# Sweetness Prediction using lasso and ridge

In [58]:
# loading libraries 
library(glmnet)
library(caTools)
set.seed(100)

In [59]:
# Reading data
sp <- read.csv("C:/Users/Shubham Shukla/Desktop/internship/new/sweetness_prediction/ecfp.csv")
sp1 <- read.csv("C:/Users/Shubham Shukla/Desktop/internship/new/sweetness_prediction/new_data/ecfp_new.csv")

In [78]:
names(sp)[2] <- "logSw"

In [61]:
sp$Sweetness <- log10(sp$Sweetness)

In [81]:
sp2 <- rbind(sp,sp1)

In [86]:
dim(sp2)

[1]  510 2049

In [87]:
# Splitting sp data into training and testing set

split <- sample.split(sp2$logSw, SplitRatio = 0.75)
train <- subset(sp2, split == T)
test <- subset(sp2, split == F)

In [88]:
train <- subset(train, select = -c(smiles))

In [89]:
test <- subset(test, select = -c(smiles))

In [90]:
testx <- model.matrix(logSw ~. -1, data = test)
testy <- test$logSw

In [91]:
# Converting into matrix 

x <- model.matrix(logSw ~. -1, data = train)
y <- train$logSw

### Using lasso

In [69]:
cv.lasso <- cv.glmnet(x,y, alpha = 1, nlambda = 100)
# plot(cv.lasso)

In [70]:
# Model fitting using lasso 

fit <- glmnet(x,y, alpha = 1, lambda = cv.lasso$lambda.1se)

In [71]:
fit


Call:  glmnet(x = x, y = y, alpha = 1, lambda = cv.lasso$lambda.1se) 

     Df   %Dev  Lambda
[1,] 17 0.9698 0.08384

In [72]:
model.predict <- predict(fit,testx)
nrow(model.predict)

[1] 44

In [73]:
sse <- sum((model.predict - testy)^2)
sst <- sum((testy - mean(testy))^2)

r_sq <- 1 - (sse/sst)

In [74]:
r_sq

[1] 0.9622437

# Using different dataset for testing

In [94]:
#Converting into matrix for lasso

data_new <- subset(sp1,select = -c(smiles))

sam <- sample.split(data_new$logSw, SplitRatio = 0.50)
sam_x <- subset(data_new, sam == T)
data_new_x <- model.matrix(logSw ~.-1, data = sam_x)
data_new_y <- sam_x$logSw

In [95]:
log.predict_new_data <- predict(log.fit, data_new_x)

In [100]:
new_sse <- sum((data_new_y - log.predict_new_data)^2)
new_sst <- sum((data_new_y - mean(data_new_y))^2)

(new_sse - new_sst)/new_sse

[1] 0.8103132